In [ ]:
import threading

import dash
import plotly.graph_objects as go
from dash import Input, Output, State, dcc, html

from geodesic import create_solver, find_path
from head_points import find_reference_points
from utils import (
    create_line,
    create_mesh,
    filter_vertices,
    path_distance,
    read_mesh,
    read_ply,
    remove_unreferenced_indices,
)

In [ ]:
points, vertices = read_mesh("data.txt")


points, vertices = filter_vertices(points, vertices)
points, vertices = remove_unreferenced_indices(points, vertices)


mesh = create_mesh(points, vertices)
solver = create_solver(points, vertices)

fig = go.Figure(data=[mesh])
fig.update_layout(
    autosize=False,
    width=800,
    height=800,
)

### find reference points (at axes intersections)

In [ ]:
all_ref_points = find_reference_points(mesh)

In [ ]:
for i in all_ref_points:
    fig.add_trace(
        go.Scatter3d(
            x=[mesh.x[i]],
            y=[mesh.y[i]],
            z=[mesh.z[i]],
            mode="markers",
            name=f"ref point {i}",
            marker=dict(size=3, color="red", opacity=0.8),
        )
    )

In [ ]:
fig

# Dash app

In [ ]:
# global variable and lock for thread safety
global_click_data = None
lock = threading.Lock()

In [ ]:
app = dash.Dash()


app.layout = dash.html.Div(
    [
        dcc.Graph(id="head-plot", figure=fig),
        html.Pre(
            id="click-data",
            style={
                "backgroundColor": "white",  # Set background color to white
                "padding": "10px",  # Optional: Add some padding
                "border": "1px solid black",  # Optional: Add a border
                "borderRadius": "5px",  # Optional: Round the corners
            },
        ),
    ]
)


@app.callback(
    Output("head-plot", "figure"),
    Output("click-data", "children"),
    Input("head-plot", "clickData"),
    State("head-plot", "figure"),
    State("head-plot", "relayoutData"),  # Capture current view settings
)
def update_plot(clickData, existing_figure, relayoutData):
    global global_click_data
    if clickData is not None:
        with lock:
            coords = clickData["points"][0]
            clicked_index = coords["pointNumber"]
            print(clicked_index)
            global_click_data = (
                coords["pointNumber"],
                (coords["x"], coords["y"], coords["z"]),
            )

            # Replace the 'Clicked Point' in the plot
            new_point = go.Scatter3d(
                x=[coords["x"]],
                y=[coords["y"]],
                z=[coords["z"]],
                mode="markers",
                marker=dict(size=5, color="blue"),
                name="Clicked Point",
                hoverinfo="none",
            )

            # Remove the last clicked point if it exists
            existing_figure["data"] = [
                trace
                for trace in existing_figure["data"]
                if trace["name"] != "Clicked Point"
            ]
            existing_figure["data"].append(new_point)

            # Apply the captured view settings to maintain orientation
            if relayoutData and "scene.camera" in relayoutData:
                existing_figure["layout"]["scene"]["camera"] = relayoutData[
                    "scene.camera"
                ]

            return (
                existing_figure,
                "Clicked coordinates: x: {:.2f}, y: {:.2f}, z: {:.2f}".format(
                    coords["x"], coords["y"], coords["z"]
                ),
            )
    return existing_figure, "Click on a point in the plot"


app.run_server(debug=True, use_reloader=False)  # Turn off reloader if inside Jupyter

In [ ]:
global_click_data

## Calculate distance from ref points (Graph and edges only)

In [ ]:
# coordinates of the index 14538 (top of the head)
ref0 = mesh.x[all_ref_points[0]], mesh.y[all_ref_points[0]], mesh.z[all_ref_points[0]]
ref0

In [ ]:
app_dist = dash.Dash()


app_dist.layout = dash.html.Div([dcc.Graph(id="head-plot", figure=fig)])


@app_dist.callback(
    Output("head-plot", "figure"),
    Input("head-plot", "clickData"),
    State("head-plot", "figure"),
    State("head-plot", "relayoutData"),  # Capture current view settings
)
def update_figure(clickData, existing_figure, relayoutData):
    if clickData is not None:
        clicked_point = clickData["points"][0]
        coords = (clicked_point["x"], clicked_point["y"], clicked_point["z"])

        clicked_index = clicked_point["pointNumber"]
        print(clicked_index)

        path_pts = find_path(solver, v_start=all_ref_points[0], v_end=clicked_index)
        line = create_line(path_pts)

        distance = path_distance(path_pts)

        # Add the distance as an annotation near the last clicked point
        annotation = {
            "text": f"Distance: {distance:.2f}",
            "xref": "paper",  # Use 'paper' for positioning relative to the entire plot
            "yref": "paper",
            "x": 0.05,  # X position in paper coordinates (0 is left, 1 is right)
            "y": 0.95,  # Y position in paper coordinates (0 is bottom, 1 is top)
            "showarrow": False,  # No arrow needed
            "font": {"size": 12},
            "bgcolor": "white",  # Background color for better visibility
            "bordercolor": "black",
            "borderwidth": 1,
        }
        existing_figure["layout"]["annotations"] = [annotation]

        clicked_point_trace = go.Scatter3d(
            x=[coords[0]],
            y=[coords[1]],
            z=[coords[2]],
            mode="markers",
            marker=dict(size=5, color="blue"),
            name="Clicked Point",
        )

        # Remove the last clicked point and edges
        existing_figure["data"] = [
            trace
            for trace in existing_figure["data"]
            if trace["name"] not in ["Clicked Point", "edge"]
        ]
        existing_figure["data"].append(clicked_point_trace)

        existing_figure["data"].append(line)

        # Apply the captured view settings to maintain orientation
        if relayoutData and "scene.camera" in relayoutData:
            existing_figure["layout"]["scene"]["camera"] = relayoutData["scene.camera"]

        return existing_figure
    return existing_figure


app_dist.run_server(
    debug=True, use_reloader=False
)  # Turn off reloader if inside Jupyter